In [8]:
import numpy as np
import open3d as o3d


'''
Thank you to the Building Parser Team from Stanford who provided the Dataset:

URL: http://buildingparser.stanford.edu/dataset.html

@ARTICLE{2017arXiv170201105A,
   author = {{Armeni}, I. and {Sax}, A. and {Zamir}, A.~R. and {Savarese}, S.
	},
    title = "{Joint 2D-3D-Semantic Data for Indoor Scene Understanding}",
  journal = {ArXiv e-prints},
archivePrefix = "arXiv",
   eprint = {1702.01105},
 primaryClass = "cs.CV",
 keywords = {Computer Science - Computer Vision and Pattern Recognition, Computer Science - Robotics},
     year = 2017,
    month = feb,
   adsurl = {http://adsabs.harvard.edu/abs/2017arXiv170201105A},
  adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}
'''

dataset_selector = "office_1"
data = np.loadtxt(f"../utils/dataset/{dataset_selector}.txt")

In [9]:
def displayVoxelCloud(cloud):
    pointCloudMatrix = np.asarray(cloud)
    pointCloudMatrix.shape
    colorMatrix = pointCloudMatrix[:, 3:6]
    pointCloudMatrix= pointCloudMatrix[:,0:3]
    pointCloudMatrix.shape
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
    
    o3dPointCloud.colors = o3d.utility.Vector3dVector(np.divide(colorMatrix, 255))
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
   # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001)
    o3d.visualization.draw_geometries([vox_mesh]) #visualization

In [10]:
displayVoxelCloud(data)